What needs to be done?

Main ingredients: "cohort", "basics", "icd9", "drugs", "lab_tests", "vital_signs"

- cohort: 

In [1]:
import psycopg2
import pandas as pd

In [2]:
sqluser = 'postgres'
sqlpass = 'postgres'
dbname = 'mimic'
schema_name = 'mimiciii'
host = 'localhost'

def run_query(sql, conn):
    query_schema = f"set search_path to public,{schema_name};"
    cur = conn.cursor()
    sql = query_schema + sql
    try:
        cur.execute(sql)
    except:
        conn.rollback()
        raise
    finally:
        cur.close()

In [3]:
def make_view_sql(sql_name, view_type="materialized view"):
    with open(f"../sql/{sql_name}.sql") as f:
        script = f.read().replace(";","")
    command = f"""drop {view_type} if exists {sql_name} cascade;
        create {view_type} {sql_name} as (
            {script}
        )"""
    return command

In [4]:
conn = psycopg2.connect(dbname=dbname, user=sqluser, password=sqlpass, host=host)

In [5]:
# generate population view
cmd = make_view_sql("population", view_type="view")
run_query(cmd, conn)

In [7]:
m_views = ("cohort", "basics", "icd9", "drugs", "lab_tests", "vital_signs")
for m in m_views:
    cmd = make_view_sql(m)
    run_query(cmd, conn)
    print("created materialized view for f{m}")